<a href="https://colab.research.google.com/github/vohoaidanh/AIGCDetectBenchmark/blob/main/colab/cross_attention_enhanced_dual_stream_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Prepare source code

In [ ]:
!git clone https://github.com/zoie-ui/AI-Generated-Image-Detection.git

Cloning into 'AI-Generated-Image-Detection'...
remote: Enumerating objects: 66, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 66 (delta 26), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (66/66), 22.64 KiB | 3.77 MiB/s, done.
Resolving deltas: 100% (26/26), done.


In [ ]:
!du -h /content/drive/MyDrive/DATASETS/T2I/T2I_origin

## Download dataset

```
# This is formatted as code
```



In [ ]:
# Move dataset to colab
import shutil
shutil.copytree('/content/drive/MyDrive/DATASETS/T2I/T2I_origin', '/content/dataset')

In [ ]:
!unzip /content/drive/MyDrive/DATASETS/Common/real_gen_dataset.zip -d /content/real_gen_dataset

## Cteate dataset *.txt files

In [ ]:
import os
based_dir = '/content/AI-Generated-Image-Detection/real_gen_dataset'
#create $_set.txt
_sets = ['val','train','test']
for _set in _sets:
  all_img = []
  imgs = []
  for a,b,c in os.walk(based_dir + f'/{_set}'):
      if 'fake' in a:
        dir = '/'.join(a.split('/')[-3:])
        print('fake',a)
        if len(c)>0:
          imgs = [os.path.join(dir,f) + " 0" for f in c]
          all_img.extend(imgs)


      if 'real' in a:
        print('real',a)
        if len(c)>0:
          dir = '/'.join(a.split('/')[-3:])
          imgs = [os.path.join(dir,f) + " 1" for f in c]
          all_img.extend(imgs)



  with open(f"{_set}.txt", "w") as file:
      # Iterate over the list
      for item in all_img:
          # Write each item to the file
          file.write(item + "\n")

real /content/AI-Generated-Image-Detection/real_gen_dataset/train
fake /content/AI-Generated-Image-Detection/real_gen_dataset/train/1_fake
real /content/AI-Generated-Image-Detection/real_gen_dataset/train/1_fake
real /content/AI-Generated-Image-Detection/real_gen_dataset/train/0_real
real /content/AI-Generated-Image-Detection/real_gen_dataset/test
fake /content/AI-Generated-Image-Detection/real_gen_dataset/test/1_fake
real /content/AI-Generated-Image-Detection/real_gen_dataset/test/1_fake
real /content/AI-Generated-Image-Detection/real_gen_dataset/test/0_real


## Train

In [ ]:
# We should to move all *.txt file dataset to txt folder below path
!mkdir /content/AI-Generated-Image-Detection/txt

In [ ]:
%cd /content/AI-Generated-Image-Detection

/content/AI-Generated-Image-Detection


In [ ]:
#Install missing package
!pip install einops -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 2.2 MB/s eta 0:00:00


In [ ]:
!python train.py \
--checkpoints /content/AI-Generated-Image-Detectioncheckpoints/checkpoints \
--batch_size 16 \
-lr 2e-4 \
--epoch 120 \
-type DALLE \



2024-04-11 04:22:23.145784: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-11 04:22:23.145834: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-11 04:22:23.147172: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-11 04:22:24.278154: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
5374
1344
[2024-04-11 04:22:25] [INFO] Namespace(checkpoints='/content/AI-Generated-Image-Detectioncheckpoints/checkpoints', batch_size=16, epoch=120, lr=0.0002, type='DALLE', size=256, split='split0', decay=30, decay_rate=0.1)
[2024-04-11 0

## Edit Mydataset.py  


```
# This is formatted as code
import torch
import torchvision.datasets as datasets
from torchvision import transforms
from torch.utils.data import Dataset
import cv2
# import PIL.Image as Image
from PIL import Image, ImageOps, ImageEnhance
import numpy as np
import random
import os
import imutils

class MyDataset(Dataset):
    def __init__(self, txt, trans=None, option=None, type=None, size=None, augment=None):
        self.root = txt
        assert(type!=None)
        assert(size!=None)
        self.option = option
        if type == "DALLE":
            if option is not None:
                self.Neg_root = os.path.join("/data/xiziyi/DALLE/DA_"+ str(size) + "_JPG_postpro/DALLE/", option) + '/'
                self.Pos_root = os.path.join("/data/xiziyi/DALLE/DA_" + str(size) + "_JPG_postpro/ALASKA/", option) + '/'
            else:
                self.Neg_root = "/data/xiziyi/DALLE/DALLE_" + str(size) + "_JPG/"
                self.Pos_root = "/data/xiziyi/DALLE/ALASKA_" + str(size) + "_JPG/"
        elif type == "DreamStudio":
            if option is not None:
                self.Neg_root = os.path.join("/data/xiziyi/ds/DS_"+ str(size) +"_JPG_postpro/DS/", option) + '/'
                self.Pos_root = os.path.join("/data/xiziyi/ds/DS_"+ str(size) + "_JPG_postpro/ALASKA/", option) + '/'
            else:
                self.Neg_root = "/data/xiziyi/ds/DS_" + str(size) + "_JPG/"
                self.Pos_root = "/data/xiziyi/DALLE/ALASKA_"+ str(size) + "_JPG/"
        elif (type == "SPL2018" or type == 'DsTok') and option is not None:
            self.Neg_root = "/data/xiziyi/" + type + '/' + "split0/224_JPG_postpro/" + option + '/' + 'CG/'
            self.Pos_root = "/data/xiziyi/" + type + '/' + "split0/224_JPG_postpro/" + option + '/' + 'PG/'
            #print(self.Neg_root)
            #print(self.Pos_root)
        self.type = type
        self.image_list = self.get_dataset_info()
        self.augment = augment
        random.shuffle(self.image_list)
        self.tensor = transforms.ToTensor()
        if transforms:
            self.transforms = trans

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, item):
        img_path, label = self.image_list[item]
        image = Image.open(img_path).convert('RGB')
        image = image.resize((256, 256))

        #resize 256 or center crop
        '''
        if self.augment:
            method = random.randint(0, 7)
            #print(method)
            if method == 1:
                num = round(random.uniform(0.5, 2.5), 2)
                image = ImageEnhance.Color(image).enhance(num)
            elif method == 2:
                num = round(random.uniform(0.5, 2.5), 2)
                image = ImageEnhance.Brightness(image).enhance(num)
            elif method == 3:
                num = round(random.uniform(0.5, 2.5), 2)
                image = ImageEnhance.Contrast(image).enhance(num)
            elif method == 4:
                num = random.randint(0, 4)
                image = ImageEnhance.Sharpness(image).enhance(num)
            elif method == 5:
                img = cv2.cvtColor(np.asarray(image), cv2.COLOR_RGB2BGR)
                angle = random.randint(-180, 180)
                image = Image.fromarray(cv2.cvtColor(imutils.rotate(img, angle), cv2.COLOR_BGR2RGB))
            elif method == 6:
                img = cv2.cvtColor(np.asarray(image), cv2.COLOR_RGB2BGR)
                num = random.randint(0, 1)
                if num == 0:
                    img = cv2.GaussianBlur(img, (5, 5), 0)
                elif num == 1:
                    img = cv2.GaussianBlur(img, (3, 3), 0)
                image = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
            elif method == 7:
                img = cv2.cvtColor(np.asarray(image), cv2.COLOR_RGB2BGR)
                num = random.randint(0, 1)
                if num == 0:
                    img = cv2.blur(img, (5, 5), 0)
                elif num == 1:
                    img = cv2.blur(img, (3, 3), 0)
                image = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        if self.type != "DALLE" or self.type != "DreamStudio":
            image = self.tensor(image)

        if image.width != 256 or image.height != 256:
            padding_w = 0
            padding_h = 0
            if image.width < 256:
                padding_w = ceil((256 - image.width) / 2)
            if image.height < 256:
                padding_h = ceil((256 - image.height) / 2)
            if padding_h != 0 or padding_w != 0:
                image = ImageOps.expand(image, (padding_w, padding_h, padding_w, padding_h))
            #image = cv2.copyMakeBorder(image, padding_h, padding_h, padding_w, padding_w,cv2.BORDER_REFLECT)
        startx = (image.height - 256) // 2
        starty = (image.width - 256) // 2
        image = image.crop((starty, startx, starty+256,startx+256))
        '''
        image = self.transforms(image)
        if label == "0":
            label = 0
        else:
            label = 1
        #torch.as_tensor(image)
        label = torch.as_tensor(label)
        return image, label

    def get_dataset_info(self):
        image_list = []
        with open(self.root,'r') as file:
            lines = file.read().splitlines()
        if (self.type == 'SPL2018' or self.type == 'DsTok') and (self.option is None):
            for line in lines:
                image_path = line[:-2]
                label = line[-1]
                image_list.append([image_path, label])
        else:
            #if self.type == 'DALLE' or self.type == 'DreamStudio':
            for line in lines:
                name = os.path.basename(line[:-2])
                #print(name)
                #if '.JPG' in name:
                #    name = name.replace('.JPG', '.jpg')
                # image_path.replace("openai_90","openai_75to85")
                label = line[-1]
                impath = line.split(' ')[0]
                # 0: dalle
                if label == '0':
                    image_path = line[:-2]# self.Neg_root + name
                # 1: alaska
                else:
                    image_path = line[:-2]# self.Pos_root + name
                image_list.append([image_path, label])
        return image_list
```



## Edit train.py



```
# This is formatted as code
import math
import torch
import torch.nn as nn
from torch.nn import init
import functools
from torch.autograd import Variable
from torch.optim import lr_scheduler
import torch.nn.functional as F
from torchvision import transforms, datasets, utils
# import matplotlib.pyplot as plt
import numpy as np
import torch.optim as optim
import os
import json
import time
import warnings
import torchvision.models as models
import torch.utils.model_zoo as model_zoo
from PIL import Image
from PIL import ImageFile

# from models.baseline.Zhang import HcNet
# from models.baseline.QuanNet import *
# import your model
from models.DualNet import *
import argparse
import logging
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
from utils.utils import Logger
import pprint
from utils.loss import Loss_Function
import torchvision
from torchsummary import summary
from utils.MyDataset import *
import shutil


def parse_args():
    parse = argparse.ArgumentParser(description='XNet')
    # the path of inpainting images
   # parse.add_argument('-img_path', '--image_path', default= "/data/xiziyi/CG/SPL2018/raw/", type=str)
    # the path of masks
    #parse.add_argument('-msk_path', '--mask_path', default='/data/xiziyi/Inpainting/Mask_4w/', type=str)
    # the path of flist ./Flist/train.txt ./Flist/val.txt ./Flist/test.txt
    #parse.add_argument('-flist', default='./Flist/', type=str)
    # the path of save log and model params
    parse.add_argument('-c', '--checkpoints', default='/data/xiziyi/paper/base/resnet18/DA_256/')
    # the params of train
    parse.add_argument('-bs', '--batch_size', default=128, type=int, required=False)
    parse.add_argument('-e', '--epoch', default=120, type=int, required=False)
    # the learning rate
    parse.add_argument('-lr', default=2e-4, type=float, required=False)
    #parse.add_argument('-model', default=DualNet, required=False)
    parse.add_argument('-type', default="DALLE", required=False)
    parse.add_argument('-size', default=256, type=int, required=False)
    parse.add_argument('-split', default="split0", required=False)
    parse.add_argument('-decay', default=30, type=int, required=False)
    parse.add_argument('-decay_rate', default=0.1, type=float, required=False)
    #parse.add_argument('--size', default=128, type=int, required=False)
    # parse.add_argument('-optim',default='Adam',choices=['Adam','SGD'])
    args = parse.parse_args()
    return args


args = parse_args()
ImageFile.LOAD_TRUNCATED_IMAGES = True
#os.environ['CUDA_VISIBLE_DEVICES'] = '1'

if args.type == "DreamStudio":
    txt1 = './Flistds/' + args.split + '/'
elif args.type == 'DALLE':
    #txt1 = './FlistDA/' + args.split + '/'
    txt1 = r'txt/'
elif args.type == 'DsTok':
    txt1 = "./FlistDsTok/" + args.split + '/'
elif args.type == 'SPL2018':
    txt1 = "./FlistSPL2018/" + args.split + '/'

class Rgb2Gray(object):
    def __init__(self):
        '''rgb_image = object.astype(np.float32)
        if len(rgb_image.shape)!= 3 or rgb_image.shape[2] != 3:
            raise ValueError("input image is not a rgb image!")'''

    def __call__(self, img):
        L_image = img.convert('L')
        return L_image

data_transform = {
    "DALLE": transforms.Compose([
            transforms.ToTensor()
         ]),
    "DreamStudio": transforms.Compose([
            transforms.ToTensor()
         ]),
    "DsTok": transforms.Compose([
            transforms.CenterCrop(args.size),
            transforms.ToTensor()
        ]),
    "SPL2018": transforms.Compose([
            transforms.CenterCrop(args.size),
            transforms.ToTensor()
        ])
}

log_dir = os.path.join(args.checkpoints + 'tensorboard', 'train')
train_writer = SummaryWriter(log_dir=log_dir)
logger_test = Logger('test', args.checkpoints + 'test_log.log')

train_dataset = MyDataset(txt=txt1 + 'train.txt',
                          trans=data_transform[args.type],
                          type=args.type,
                          size=args.size)
train_num = len(train_dataset)
print(train_num)
train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=args.batch_size,
                                           shuffle=True,
                                           drop_last=True)

validate_dataset = MyDataset(txt=txt1 + 'val.txt',
                             trans=data_transform[args.type],
                             type=args.type,
                             size=args.size)
val_num = len(validate_dataset)
print(val_num)
validate_loader = torch.utils.data.DataLoader(validate_dataset,
                                              batch_size=args.batch_size,
                                              shuffle=True,
                                              drop_last=True)

# test
def train():
    # save_path /data/xiziyi/project/XNet/checkpoints/UNet/
    if not os.path.exists(args.checkpoints):
        os.makedirs(args.checkpoints)
    logger = Logger('train', args.checkpoints + 'train_log.log')
    # logger1 = logging.getLogger('train')
    # writer = SummaryWriter(log_dir=os.path.join(args.checkpoints,'result-logs'))
    logger.info(pprint.pformat(args))
    logger.info(
        #f"model: {args.model} | "
        f"mkdir: {txt1} | "
        f"dataset: {args.type} | "
        f"input_size: {args.size} |\n"
    )
    #net = models.resnet18(pretrained=False)
    net = DualNet()
    net = net.cuda()
    net = nn.DataParallel(net)
    logger.info(net)
    optimizer = optim.Adam(net.parameters(), lr=args.lr)
    #optimizer = optim.SGD(net.parameters(),lr=args.lr,momentum=0.9,weight_decay=1e-4)
    # step_size: the time of involved
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=args.decay, gamma=args.decay_rate)
    logger.info(pprint.pformat(optimizer.state_dict()['param_groups'][0]))
    # optimizer = optim.SGD(net.parameters(), lr=1e-3, momentum=0.9, weight_decay=1e-4)
    # new_lr = 1e-4
    # sigmoid + BCE_loss
    best_loss = 10.0
    best_acc = 0.0
    loss_function = Loss_Function()
    for epoch in range(args.epoch):
        # if epoch % 20 == 0:
        #    new_lr = new_lr * 0.8
        #    optimizer = optim.Adam(net.parameters(), lr=new_lr)
        net.train()
        running_loss = 0.0
        running_corrects = 0.0
        time_start = time.perf_counter()
        warnings.filterwarnings('ignore')
        print('epoch[%d]  ' % epoch)
        step = 0

        for data in tqdm(train_loader, dynamic_ncols=True):
            images, labels = data
            images = images.cuda()
            labels = labels.cuda()
            optimizer.zero_grad()
            outputs = net(images)
            preds = torch.max(outputs, dim=1)[1]
            # print(outputs.size())
            # print(labels.size())
            loss = loss_function(outputs, labels)
            # print("loss:",loss)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            running_corrects += (preds == labels).sum().item()
            step += 1
            # running_corrects += (preds == labels.cuda()).sum().item()

        scheduler.step()
        # writer.add_scalar('train/loss', running_loss/step, epoch)
        train_loss = running_loss / step
        train_acc = running_corrects / train_num
        logger.info(
            f"Train epoch {epoch}: "
            f" Time waste: {time.perf_counter() - time_start} |"
            # f"loss: {train_loss_meter['loss'].avg:.6f} |"
            f" loss: {train_loss:.6f} |"
            f" train_acc: {train_acc:.6f} |"
            # f"loss_pred: {train_loss_meter['loss_pred'].avg:.6f} |"
            f" lr: {optimizer.state_dict()['param_groups'][0]['lr']:.6f} |"
            # f"f1: |"
            # f"mIoU: |"
        )

        ########################################### validate ###########################################

        # net = torch.load('Result2.pth')
        net.eval()
        val_loss_sum = 0.0
        acc = 0.0
        with torch.no_grad():
            step = 0
            for val_data in tqdm(validate_loader):
                val_images, val_labels = val_data
                val_images = val_images.cuda()
                val_labels = val_labels.cuda()
                outputs = net(val_images)
                loss1 = loss_function(outputs, val_labels)
                result = torch.max(outputs, dim=1)[1]
                val_loss_sum += loss1.item()
                acc += (result == val_labels.cuda()).sum().item()
                step += 1
            val_loss = val_loss_sum / step
            val_acc = acc / val_num
            logger.info(
                f"Val epoch {epoch}: "
                f" loss: {val_loss:.3f} |"
                f" val_acc: {val_acc:.3f} |\n"
            )
            train_writer.add_scalars("loss", {"train_loss": train_loss, "val_loss": val_loss, "train_acc": train_acc,
                                              "val_acc": val_acc}, epoch)
            if val_acc > best_acc:
                best_acc = val_acc
                # best_loss = val_loss / val_num
                torch.save(
                    {'net': net.module.state_dict(),
                     'opt': optimizer.state_dict(),
                     'epoch': epoch,
                     'loss': val_loss,
                     'acc': best_acc}, args.checkpoints + 'model_best_acc.pt'
                )
            if val_loss < best_loss:
                best_loss = val_loss
                torch.save(
                    {'net': net.module.state_dict(),
                     'opt': optimizer.state_dict(),
                     'epoch': epoch,
                     'loss': best_loss,
                     'acc': val_acc
                     }, args.checkpoints + 'model_best_loss.pt'
                )
    print('Finished Training')

def test(name, op=None):
    # print(image_path + '/' +
    # logger_test.info(summary(net,))
    test_dataset = MyDataset(txt=txt1 + 'test.txt',
                             trans=data_transform[args.type],
                             option=op,
                             type=args.type,
                             size=args.size)
    test_num = len(test_dataset)
    print(test_num)
    test_loader = torch.utils.data.DataLoader(test_dataset,
                                              batch_size=args.batch_size,
                                              shuffle=True,
                                              drop_last=False)
    net = DualNet()
    '''
    dicts = torch.load(name)['net']
    mynet_dict = net.state_dict()
    new_dict = {k: v for k, v in dicts.items() if k in mynet_dict.keys()}
    mynet_dict.update(new_dict)
    net.load_state_dict(mynet_dict)
    '''
    #net.load_state_dict(torch.load(name)['net'])
    net.load_state_dict(torch.load(name)['net'])
    net = nn.DataParallel(net)
    net.cuda()
    print("model has been imported successfully!")

    net.eval()
    loss_function = nn.CrossEntropyLoss()
    test_loss = 0.0
    acc = 0.0
    TP = 0.0
    TN = 0.0
    FP = 0.0
    FN = 0.0
    time_start = time.perf_counter()
    with torch.no_grad():
        for step, test_data in enumerate(test_loader, start=0):
            #print(test_data)
            test_images, test_labels = test_data
            test_images = test_images.cuda()
            test_labels = test_labels.cuda()
            outputs = net(test_images)
            b, _ = outputs.size()
            # bsx1xhxw
            result = torch.max(outputs, dim=1)[1]
            for i in range(b):
                if result[i] == test_labels[i] and test_labels[i] == 0:
                    TN += 1
                if result[i] == test_labels[i] and test_labels[i] == 1:
                    TP += 1
                if result[i] != test_labels[i]:
                    if result[i] == 1:
                        #target_pth = target1 + os.path.basename(img_path[i])
                        #shutil.copyfile(img_path[i], target_pth)
                        FP += 1
                    else:
                        #target_pth = target2 + os.path.basename(img_path[i])
                        #shutil.copyfile(img_path[i], target_pth)
                        FN += 1
            loss1 = loss_function(outputs, test_labels)
            test_loss += loss1.item()
        TPR = TP / (TP+FN)
        TNR = TN / (TN+FP)
        test_acc = (TP+TN) / test_num
        logger_test.info(
            f"[dataset {args.type}]: {args.size}x{args.size} "
            f"time_waste:{time.perf_counter() - time_start:.3f} "
            f"test: {op} |"
            f"TPR: {TPR:.4f} |"
            f"TNR: {TNR:.4f} |"
            f"loss: {(test_loss / step):.4f} |"
            f"acc: {test_acc:.4f} |"
        )


if __name__ == '__main__':
    #bacc = args.checkpoints + 'DsTok.pth'
    train()
    bacc = args.checkpoints + 'model_best_acc.pt'
    bloss = args.checkpoints + 'model_best_loss.pt'
    test(bacc)
    test(bloss)
    #test(bacc, "rotate")
    #test(bloss, "rotate")
    #options = ["gb3x3", "gb5x5", "median3x3", "median5x5", "avg3x3", "avg5x5", "resize0.5", "resize0.8", "test_75", "test_85", "test_95"]
    #options = ['color', 'brightness', 'contrast', 'sharp', "rotate", "gb5x5", "avg5x5"]
    #options = ["gb3x3", "gb5x5"]
    #for op in options:
    #    test(bacc, op)

    #for op in options:
    #    test(bloss, op)

```

